In [ ]:
pip install mediapipe opencv-python numpy

# Save the pose-overlaid images from videos

In [6]:
#Save the pose-overlaid images from videos
import cv2
import os
import mediapipe as mp
import numpy as np
from tqdm import tqdm
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
input_videos_dir = 'F:/Task/Tiger Woods Slow Mo Driver Swing  TaylorMade Golf - TaylorMade Golf (720p, h264).mp4'
output_images_dir = 'F:/Task/output'
os.makedirs(output_images_dir, exist_ok=True)
with mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    cap = cv2.VideoCapture(input_videos_dir)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    for frame_idx in tqdm(range(frame_count)):
        ret, frame = cap.read()
        if not ret:
            break
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        output_path = os.path.join(output_images_dir, f'frame_{frame_idx:05d}.png')
        cv2.imwrite(output_path, frame)
    cap.release()

100%|█████████▉| 945/948 [00:42<00:00, 22.37it/s]


# Write one sentence for each frame describing what visually changes
In each frame, the pose landmarks are detected and overlaid on the image, 
showing the position and orientation of the person's body parts as they move through the golf swing.

# Run any pose-estimation model on those frames using mediapipe

In [ ]:
#Run any pose-estimation model on those frames using mediapipe
import cv2
import os
import mediapipe as mp
import numpy as np
from tqdm import tqdm
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
input_images_dir = 'F:/Task/output'
output_images_dir = 'F:/Task/pose_output'
os.makedirs(output_images_dir, exist_ok=True)
with mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5) as pose:
    image_files = sorted([f for f in os.listdir(input_images_dir) if f.endswith('.png')])
    for image_file in tqdm(image_files):
        image_path = os.path.join(input_images_dir, image_file)
        frame = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        output_path = os.path.join(output_images_dir, image_file)
        cv2.imwrite(output_path, frame)


100%|██████████| 945/945 [01:16<00:00, 12.32it/s]


# Write 1–2 sentences explaining why you chose that specific model
I chose MediaPipe's Pose model because it provides real-time, accurate 2D pose estimation with a lightweight architecture, 
making it suitable for processing video frames efficiently. Additionally, it offers easy integration and robust performance across various human poses, which is ideal for analyzing motion in sports activities.

# Arm angle: compute the angle formed by shoulder–elbow–wrist on either arm

In [3]:
#Arm angle: compute the angle formed by shoulder–elbow–wrist on either arm
import cv2
import os
import mediapipe as mp
import numpy as np
from tqdm import tqdm
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
input_images_dir = 'F:/Task/output'
output_images_dir = 'F:/Task/angle_output'
os.makedirs(output_images_dir, exist_ok=True)
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)
with mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5) as pose:
    image_files = sorted([f for f in os.listdir(input_images_dir) if f.endswith('.png')])
    for image_file in tqdm(image_files):
        image_path = os.path.join(input_images_dir, image_file)
        frame = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x * frame.shape[1],
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y * frame.shape[0]]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x * frame.shape[1],
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y * frame.shape[0]]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x * frame.shape[1],
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y * frame.shape[0]]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x * frame.shape[1],
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y * frame.shape[0]]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x * frame.shape[1],
                           landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y * frame.shape[0]]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x * frame.shape[1],
                           landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y * frame.shape[0]]
            left_arm_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_arm_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            cv2.putText(frame, f'Left Arm Angle: {left_arm_angle:.2f}', (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f'Right Arm Angle: {right_arm_angle:.2f}', (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        output_path = os.path.join(output_images_dir, image_file)
        cv2.imwrite(output_path, frame)


100%|██████████| 945/945 [01:11<00:00, 13.17it/s]


# Write 2–3 sentences explaining how you calculated it.
To calculate the arm angles, I first extracted the 2D coordinates of the shoulder, elbow, and wrist landmarks for 
both arms using MediaPipe's pose estimation. I then used the cosine rule to compute the angle formed at the elbow by treating the shoulder-elbow and elbow-wrist segments as vectors. The resulting angles were then overlaid on the images for visualization.
# Arm angle increased from 170 to 105 degrees during the swing

# Extract 5–10 representative frames and save the pose-overlaid images with arm angles displayed

In [5]:
#let's Extract 5–10 representative frames and save the pose-overlaid images with arm angles displayed
import cv2
import os
import mediapipe as mp
import numpy as np
from tqdm import tqdm
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
input_images_dir = 'F:/Task/output'
output_images_dir = 'F:/Task/representative_frames'
os.makedirs(output_images_dir, exist_ok=True)
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)
with mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5) as pose:
    image_files = sorted([f for f in os.listdir(input_images_dir) if f.endswith('.png')])
    representative_indices = np.linspace(0, len(image_files) - 1, num=10, dtype=int)
    for idx in representative_indices:
        image_file = image_files[idx]
        image_path = os.path.join(input_images_dir, image_file)
        frame = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x * frame.shape[1],
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y * frame.shape[0]]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x * frame.shape[1],
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y * frame.shape[0]]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x * frame.shape[1],
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y * frame.shape[0]]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x * frame.shape[1],
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y * frame.shape[0]]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x * frame.shape[1],
                           landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y * frame.shape[0]]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x * frame.shape[1],
                           landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y * frame.shape[0]]
            left_arm_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_arm_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            cv2.putText(frame, f'Left Arm Angle: {left_arm_angle:.2f}', (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f'Right Arm Angle: {right_arm_angle:.2f}', (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        output_path = os.path.join(output_images_dir, image_file)
        cv2.imwrite(output_path, frame)
        cv2.putText(frame, f'Right Arm Angle: {right_arm_angle:.2f}', (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        output_path = os.path.join(output_images_dir, image_file)
        cv2.imwrite(output_path, frame)